## Exercício sobre pytorch

Utilize o dataset 'datasetCarros.csv'.<br>
Usando Pytorch, construa uma rede neural para prever a feature 'PrecoVenda'.<br>

Use uma rede neural feed forward com duas camadas escondidas, com 20 neurônios cada.<br>
Use o critério de perda MSELoss, otimizador Adam e learning rate = 0.001. Considere 1000 épocas.

In [ ]:
"""
import pandas as pd
from sklearn.model_selection import train_test_split
from torch import nn
import torch
import numpy as np

class Feedforward(torch.nn.Module):
    
        def __init__(self, input_size, hidden_size):
            super(Feedforward, self).__init__()
            
            self.input_size = input_size
            self.hidden_size  = hidden_size
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
            self.fc2 = torch.nn.Linear(self.hidden_size, self.hidden_size)
            self.fc3 = torch.nn.Linear(self.hidden_size, 1)
            
            self.relu = torch.nn.ReLU()
            
            
        def forward(self, x):
            output = self.fc1(x)
            output = self.relu(output)
            
            output = self.fc2(output)
            output = self.relu(output)
            
            output = self.fc3(output)

            return output

device = "cuda" if torch.cuda.is_available() else "cpu"

dfCarros = pd.read_csv('datasetCarros.csv')

dfCarros.replace({'TipoCombustivel':{'Petrol':0,'Diesel':1,'CNG':2}},inplace=True)
dfCarros.replace({'Trasmissao':{'Manual':0,'Automatic':1}},inplace=True)


y= dfCarros['PrecoVenda']
X = dfCarros.drop(['Nome','PrecoVenda'],axis=1)


y_tensor = torch.tensor(y)
X_tensor = torch.tensor(X.to_numpy())
print(y_tensor.shape)
print(X_tensor.shape)

X_treino, X_teste, y_treino, y_teste = train_test_split(X_tensor, y_tensor, test_size = 0.10, random_state=5)

print(X_treino.shape)
print(y_treino.shape)

X_treino = X_treino.float().to(device)
y_treino = y_treino.float().to(device)

X_teste = X_teste.float().to(device)
y_teste = y_teste.float().to(device)


model = Feedforward(6, 20).to(device)
print(model)
criterion = torch.nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)


model.eval()
y_pred = model(X_teste)
antes_treino = criterion(y_pred, y_teste) 
print('Teste - perda antes do treinamento' , antes_treino.item())

model.train()
epoch = 100

for epoch in range(epoch):
    optimizer.zero_grad()
    
    # Passe Forward
    y_pred = model(X_treino)
    
    # Computa a perda
    loss = criterion(y_pred, y_treino)
    
    print('Epoch {}: perda treino: {}'.format(epoch, loss.item()))
    
    # Passe de Backward
    loss.backward()
    optimizer.step()

    
model.eval()
y_pred = model(X_teste)
after_train = criterion(y_pred, y_teste) 
print('Teste - perda depois do treinamento' , after_train.item())    
"""

Imports

In [25]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

Carregue os dados

In [26]:
data = pd.read_csv('datasetCarros.csv')

In [27]:
input_features = ['Ano', 'PrecoAtual', 'KmRodado']
target_column = 'PrecoVenda'

In [28]:
train_data = data.sample(frac=0.8, random_state=42)
test_data = data.drop(train_data.index)

In [29]:
X_train = train_data[input_features].values
y_train = train_data[target_column].values
X_test = test_data[input_features].values
y_test = test_data[target_column].values

Normalização dos dados

In [30]:
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std


tensores

In [31]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

DataLoader

In [32]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

rede neural

In [33]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

definindo modelo

In [34]:
model = NeuralNet(input_size=len(input_features), hidden_size=20)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

treinando modelo

In [35]:
num_epochs = 1000
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

c:\Users\andso\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\andso\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([17])) that is different to the input size (torch.Size([17, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [100/1000], Loss: 69.1958
Epoch [200/1000], Loss: 30.2072
Epoch [300/1000], Loss: 12.5078
Epoch [400/1000], Loss: 63.2541
Epoch [500/1000], Loss: 9.4110
Epoch [600/1000], Loss: 54.1715
Epoch [700/1000], Loss: 31.5970
Epoch [800/1000], Loss: 30.8249
Epoch [900/1000], Loss: 10.5177
Epoch [1000/1000], Loss: 14.5557


Avaliação

In [36]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    test_loss = criterion(y_pred, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')

Test Loss: 18.1737


c:\Users\andso\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([60])) that is different to the input size (torch.Size([60, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
